In [1]:
import matplotlib.pyplot as plt
import numpy as np
from filter import filter_encode, filter_decode
from helper import runlength_encode
import keras
from collections import defaultdict

from itertools import chain

from sklearn.cluster import KMeans, MiniBatchKMeans

import multiprocessing as multi
from multiprocessing import Pool

import pickle

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train, y_train), (test_images, test_labels) = fashion_mnist.load_data()

In [3]:
def freq_vector(arr):
    d = {i: 1 for i in range(256)}
    for i in arr:
        d[i] += 1
    return d

def to_freq(img):
    fil = filter_encode(img, fid=1)
    rle = runlength_encode(fil.ravel())
    return list(freq_vector(rle).values())

In [4]:
rles = []

n_cores = multi.cpu_count()
with Pool(n_cores) as p:
    freq = p.map(to_freq, X)

NameError: name 'X' is not defined

In [6]:
X = [x_train[i, :, :] for i in range(60000)]

n_cores = multi.cpu_count()
with Pool(n_cores) as p:
    freq = p.map(to_freq, X)

print(np.array(freq))

new_X = np.array(freq)

[[ 68  23  30 ...  16  19  13]
 [102  37  40 ...  23  35  31]
 [105  22  26 ...  10  18  21]
 ...
 [ 86  20  14 ...  13  10  18]
 [ 98  37  43 ...  20  19  16]
 [ 44  17   6 ...   8   3  15]]


In [7]:
new_X.shape

(60000, 256)

In [8]:
km = MiniBatchKMeans(n_clusters=256, max_iter=1000)

km.fit(new_X)
km.cluster_centers_

array([[111.8164557 ,  20.34810127,  35.67088608, ...,  27.96202532,
         29.08227848,  25.38607595],
       [ 35.59452055,  10.7369863 ,  17.96986301, ...,   9.40821918,
         10.40273973,  10.61917808],
       [ 75.12121212,  12.03030303,  14.85353535, ...,   9.26767677,
          9.62121212,  11.46969697],
       ...,
       [ 74.01877934,  17.82629108,  34.15023474, ...,  13.18779343,
         17.7370892 ,  15.86384977],
       [ 80.83414239,  23.85355987,  37.92394822, ...,  15.97168285,
         19.91666667,  22.1618123 ],
       [ 40.54285714,   1.13333333,  15.0952381 , ...,   6.95238095,
          7.39047619,   1.58095238]])

In [11]:
type(km.labels_)

numpy.ndarray

In [12]:
x_train[km.labels_ == 0]

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 1, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [13]:
from huffman import HuffmanCoding

dicts = []
trees = []

for i in range(256):
    huff = HuffmanCoding()
    label = x_train[km.labels_ == i]

    label_list = [label[i, :, :].ravel() for i in range(len(label))]

    n_cores = multi.cpu_count()
    with Pool(n_cores) as p:
        rle = p.map(runlength_encode, label_list)

    rle = chain.from_iterable(rle)
    enc = huff.encode(rle)

    dicts.append(huff.code)
    trees.append(huff.tree)



In [14]:
dicts[0]

{70: '00000000',
 205: '00000001',
 110: '00000010',
 198: '00000011',
 74: '00000100',
 203: '00000101',
 208: '00000110',
 194: '00000111',
 103: '00001000',
 217: '00001001',
 92: '00001010',
 20: '00001011',
 241: '000011000',
 123: '000011001',
 86: '00001101',
 76: '00001110',
 73: '00001111',
 210: '00010000',
 67: '00010001',
 80: '00010010',
 187: '00010011',
 140: '00010100',
 160: '00010101',
 235: '00010110',
 82: '00010111',
 155: '00011000',
 201: '00011001',
 183: '00011010',
 184: '00011011',
 190: '00011100',
 93: '00011101',
 220: '00011110',
 150: '00011111',
 100: '00100000',
 22: '00100001',
 204: '00100010',
 62: '00100011',
 133: '00100100',
 26: '00100101',
 233: '00100110',
 32: '00100111',
 193: '00101000',
 185: '00101001',
 180: '00101010',
 199: '00101011',
 106: '00101100',
 149: '00101101',
 202: '00101110',
 186: '00101111',
 47: '00110000',
 191: '00110001',
 240: '001100100',
 252: '0011001010',
 249: '0011001011',
 102: '00110011',
 64: '00110100',
 1

In [15]:
with open("dicts.pkl", "wb") as f:
    pickle.dump(dicts, f)

with open("trees.pkl", "wb") as f:
    pickle.dump(trees, f)

In [19]:
with open("trees.pkl", "rb") as f:
    trees_loaded = pickle.load(f)

print(trees_loaded)

[<huffman.Node object at 0x14bcce790>, <huffman.Node object at 0x14f0342d0>, <huffman.Node object at 0x14fcdf7d0>, <huffman.Node object at 0x14fd8cad0>, <huffman.Node object at 0x14bcca750>, <huffman.Node object at 0x14ee15910>, <huffman.Node object at 0x1504fad90>, <huffman.Node object at 0x1504bd8d0>, <huffman.Node object at 0x14fcafdd0>, <huffman.Node object at 0x15044d590>, <huffman.Node object at 0x14bcef850>, <huffman.Node object at 0x14f7fa610>, <huffman.Node object at 0x14f548890>, <huffman.Node object at 0x14f5dd710>, <huffman.Node object at 0x14fbd90d0>, <huffman.Node object at 0x14f612710>, <huffman.Node object at 0x14f80ce90>, <huffman.Node object at 0x14f047650>, <huffman.Node object at 0x14bcd6690>, <huffman.Node object at 0x14fab2710>, <huffman.Node object at 0x14f5e3590>, <huffman.Node object at 0x14f938ed0>, <huffman.Node object at 0x14bcea990>, <huffman.Node object at 0x14bce45d0>, <huffman.Node object at 0x14f104ed0>, <huffman.Node object at 0x14f790dd0>, <huffman.No

In [16]:
with open("cluster_centers.pkl", "wb") as f:
    pickle.dump(km.cluster_centers_, f)

km.cluster_centers_

array([[111.8164557 ,  20.34810127,  35.67088608, ...,  27.96202532,
         29.08227848,  25.38607595],
       [ 35.59452055,  10.7369863 ,  17.96986301, ...,   9.40821918,
         10.40273973,  10.61917808],
       [ 75.12121212,  12.03030303,  14.85353535, ...,   9.26767677,
          9.62121212,  11.46969697],
       ...,
       [ 74.01877934,  17.82629108,  34.15023474, ...,  13.18779343,
         17.7370892 ,  15.86384977],
       [ 80.83414239,  23.85355987,  37.92394822, ...,  15.97168285,
         19.91666667,  22.1618123 ],
       [ 40.54285714,   1.13333333,  15.0952381 , ...,   6.95238095,
          7.39047619,   1.58095238]])